This example shows how to use [AutoGluon](https://auto.gluon.ai) to build a solution with private LB: 16.96991 and public LB: 17.89868 (equivalent to 20rd/3537 in the competition). 

AutoGluon is an AutoML toolkit that makes it easy to build strong ML models for your application that may involve image, text and tabular data. In the recent 0.5 release, AutoGluon launched a new feature called `autogluon.multimodal`, or [AutoMM](https://auto.gluon.ai/stable/tutorials/multimodal/index.html). AutoMM is a deep learning “model zoo” of model zoos that integrates popular DL packages like [timm](https://github.com/rwightman/pytorch-image-models), [huggingface/transformers](https://github.com/huggingface/transformers), [openai/CLIP](https://github.com/openai/CLIP), etc. Users can easily build deep learning models that fuse these backbones to tackle both unimodal and multimodal problems. Details about how to train models for this competition is described in [this example](https://github.com/awslabs/autogluon/tree/master/examples/automm/kaggle_pawpularity). In short, we ensembled models with three backbones: `vit_large_patch16_384`, `swin_large_patch4_window12_384`,`convnext_large_384_in22ft2k` and `swin_large_patch4_window7_224`, the first two models are fusion models that combine the image and the metadata, while the last two only use models in timm.

For more details about AutoGluon, check our website: https://auto.gluon.ai, Github repo: https://github.com/awslabs/autogluon, or install it via `pip install autogluon`. We are actively improving the package and definitely welcome feedback and contributions!

In [ ]:
!nvidia-smi
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
sys.path.append('../input/autogluon-standalone-install/autogluon_standalone/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8/src/')
!pip install --no-deps --no-index --quiet ../input/autogluon-standalone-install/autogluon_standalone/*.whl --find-links autogluon_standalone

In [ ]:
from autogluon.multimodal import AutoMMPredictor
import pandas as pd
import numpy as np
import torch
import os


data_path = "../input/petfinder-pawpularity-score/"

config_6 = {
    "save_path": "../input/pawpularity-automm-result/result6/result",
    "per_gpu_batch_size_evaluation": 32,
    "N_fold": 5,
}
config_7 = {
    "save_path": "../input/pawpularity-automm-result/result7/result",
    "per_gpu_batch_size_evaluation": 3,
    "N_fold": 5,
}
config_13 = {
    "save_path": "../input/pawpularity-automm-result/result13/result",
    "per_gpu_batch_size_evaluation": 32,
    "N_fold": 5,
}
config_26 = {
    "save_path": "../input/pawpularity-automm-result26/result",
    "per_gpu_batch_size_evaluation": 3,
    "N_fold": 5,
}
config_30 = {
    "save_path": "../input/pawpularity-automm-result30/result",
    "per_gpu_batch_size_evaluation": 32,
    "N_fold": 5,
}


def load_data(data_path):
    train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
    train_df.rename(columns={"Id": "Image Path"}, inplace=True)
    train_df["Image Path"] = train_df["Image Path"].apply(lambda s: os.path.join(data_path, "train", s + ".jpg"))

    test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
    test_df.rename(columns={"Id": "Image Path"}, inplace=True)
    test_df["Image Path"] = test_df["Image Path"].apply(lambda s: os.path.join(data_path, "test", s + ".jpg"))
    return train_df, test_df


train_df, test_df = load_data(data_path)

if __name__ == "__main__":
    submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
    
    configs = [config_6, config_7, config_26, config_13]
    model_preds = np.empty(shape=[0,submission.shape[0]])
    for perconfig in configs:
        print(perconfig)
        save_standalone_path = perconfig["save_path"] + '_standalone'
        all_preds = []
        for fold in range(perconfig["N_fold"]):
            pretrained_model = AutoMMPredictor.load(path=save_standalone_path + f'_fold{fold}/')
            pretrained_model._config.env.per_gpu_batch_size_evaluation = perconfig["per_gpu_batch_size_evaluation"]
            df_test = pretrained_model.predict(test_df)
            all_preds.append(df_test)
            del pretrained_model
            torch.cuda.empty_cache()
        model_preds = np.append(model_preds, [np.mean(np.stack(all_preds), axis=0)], axis=0)
    submission["Pawpularity"] = model_preds[0] * 0.3 + model_preds[1] * 0.4  + model_preds[2] * 0.2 + model_preds[3] * 0.1 #Model ensemble.
    submission.to_csv("submission.csv", index=False)

    print(submission)